In [ ]:
import os
import json
import numpy as np
import pandas as pd

from tyssue import Sheet
from tyssue.io import hdf5
from tyssue.utils.utils import _to_3d, to_nd
from tyssue.dynamics.sheet_gradients import height_grad
from tyssue.dynamics import units, effectors, model_factory
from tyssue import Monolayer, MonolayerGeometry, ClosedMonolayerGeometry, SheetGeometry
from tyssue.generation import extrude


from tyssue.behaviors.event_manager import EventManager
from tyssue.behaviors.sheet.basic_events import reconnect, reconnect_3D
from tyssue.dynamics import model_factory, effectors

from tyssue.solvers import QSSolver

from tyssue.io.hdf5 import save_datasets

from tyssue.draw import sheet_view
import ipyvolume as ipv

In [ ]:
def initiate_ellipsoid(dataset_path, json_path):
    """
    Create ellipsoid tissue as a sheet with mesodermal cells
    dataset_path: initial hf45 file
    json_path: json spec file
    """
    dsets = hdf5.load_datasets(dataset_path,
                               data_names=['vert', 'edge', 'face'])

    with open(json_path, 'r+') as fp:
        specs = json.load(fp)

    sheet = Sheet('ellipse', dsets, specs)

    # Modify some initial value
    sheet.settings['threshold_length'] = 1e-3
    sheet.settings['vitelline_space'] = 0.2
    sheet.vert_df['radial_tension'] = 0.
    sheet.settings['lumen_prefered_vol'] = 4539601.384437251
    sheet.settings['lumen_vol_elasticity'] = 3.e-6
    sheet.edge_df.cell = np.nan


    return sheet

In [ ]:
sheet = initiate_ellipsoid('ellipsoid_sheet_init.hf5',
                         'ellipsoid.json')

In [ ]:
datasets = extrude(sheet.datasets, method='homotecy', scale=0.9)

monolayer = Monolayer('mono_embryo', datasets)
monolayer.vert_df['z'] += 5

ClosedMonolayerGeometry.update_all(monolayer)

ipv.clear()

fig, mesh = sheet_view(monolayer, mode="3D")
fig


In [ ]:
monolayer.face_df['prefered_area'] = monolayer.face_df['area']/5
monolayer.face_df['prefered_perimeter'] = 3 * np.sqrt(monolayer.face_df['prefered_area'])
monolayer.face_df['area_elasticity'] = 1
monolayer.face_df['perimeter_elasticity'] = 0.5
monolayer.cell_df['volume_elasticity'] = 1
monolayer.cell_df['prefered_volume'] = 1

monolayer.settings['lumen_prefered_volume'] = 4539601.384437251 *50/100
monolayer.settings['lumen_volume_elasticity'] = 3.e-6
monolayer.settings.update(sheet.settings)

In [ ]:
class VitellineElasticity(effectors.AbstractEffector):

    dimensions = units.line_elasticity
    magnitude = 'vitelline_K'
    label = 'Vitelline elasticity'
    element = 'vert'
    specs = {
        'vert': {
            'vitelline_K': 280,
            'is_active': 1.0,
            'delta_rho': 1.0}
    }  # distance to the vitelline membrane

    @staticmethod
    def energy(eptm):
        return eptm.vert_df.eval(
            'delta_rho**2 * vitelline_K/2')

    @staticmethod
    def gradient(eptm):
        grad = height_grad(eptm) * _to_3d(
            eptm.vert_df.eval('vitelline_K * delta_rho'))
        grad.columns = ['g' + c for c in eptm.coords]
        return grad, None
    
class ClosedEllipsoidGeometry(ClosedMonolayerGeometry):
    @staticmethod
    def update_height(eptm):

        a, b, c = eptm.settings["abc"]
        eptm.vert_df["theta"] = np.arcsin((eptm.vert_df.z / c).clip(-1, 1))
        eptm.vert_df["vitelline_rho"] = a * np.cos(eptm.vert_df["theta"])
        eptm.vert_df["basal_shift"] = (
            eptm.vert_df["vitelline_rho"] - eptm.specs["vert"]["basal_shift"]
        )
        eptm.vert_df["delta_rho"] = (
            np.linalg.norm(eptm.vert_df[["x", "y"]], axis=1)
            - eptm.vert_df["vitelline_rho"]
        ).clip(lower=0)

        SheetGeometry.update_height(eptm)

    @staticmethod
    def scale(eptm, scale, coords):
        SheetGeometry.scale(eptm, scale, coords)
        eptm.settings["abc"] = [u * scale for u in eptm.settings["abc"]]


In [ ]:
model = model_factory(
    [
        VitellineElasticity, 
#         effectors.LineTension,
        effectors.FaceAreaElasticity,
        effectors.PerimeterElasticity,
        effectors.CellVolumeElasticity,
        effectors.LumenVolumeElasticity,
    ],
)

# Manager
manager = EventManager('face')

In [ ]:
solver_qs = QSSolver(with_t1=False, with_t3=False, with_collisions=False)

# manager.append(reconnect_3D)
for i in range(1):
    print('------------TEMPS------------')
    print(i)
    manager.execute(monolayer)
    
    res = solver_qs.find_energy_min(monolayer, ClosedEllipsoidGeometry, model, periodic=False, options={"gtol": 1e-8})
    if res.success is False:
        print (i, res.success)
        
    monolayer.vert_df[["x", "y"]] += np.random.normal(scale=1e-3, size=(monolayer.Nv, 2))
    manager.update()
#     save_datasets(os.path.join(test,'monolayer'+str(i)+'.hf5'), monolayer)
    

In [ ]:
ipv.clear()

fig, mesh = sheet_view(monolayer, mode="3D")
fig


In [ ]:
from tyssue.io.meshes import save_triangular_mesh
save_triangular_mesh('monolayer_3d.vtk', monolayer)

In [ ]:
monolayer.vert_df